In [1]:
import os
import pandas as pd

In [2]:
# 英超使用(2018-2019)資料評估(2019-2020)賽季比分分布狀況
# TODO 資料整理
teams = pd.read_csv(os.getcwd() + '/data/82/teams.csv')
teams.set_index(["team_id"], inplace=True)

matches = pd.read_csv(os.getcwd() + '/data/82/matches.csv')
lastSeason = matches[matches['season_id'] == 388]
curSeason = matches[matches['season_id'] == 5073]

In [3]:
# TODO 計算平均進球數
# 聯盟主隊平均主場進球數 = 主場賽季總進球數/比賽場數（賽季）
seasonHomeAvg = lastSeason['home_score'].sum() / lastSeason.shape[0]
# 聯盟主隊平均客場進球數 = 客場賽季總進球數/比賽場數（賽季）
seasonAwayAvg = lastSeason['away_score'].sum() / lastSeason.shape[0]

# TODO 統計資料
# 隊伍為主場平均進球
temp = lastSeason.groupby(['home_team'])['home_score', 'away_score'].sum()
teams = teams.join(temp, on='team_id')
teams = teams.rename(columns={"home_score": "homeGetScore", "away_score": "homeLostScore"})

# 隊伍為主場總場次
temp = lastSeason.groupby(['home_team']).size()
temp.index.rename('team_id')
temp = temp.rename('homeMatchCount')
teams = teams.join(temp, on='team_id')

# 隊伍為客場平均進球
temp = lastSeason.groupby(['away_team'])['home_score', 'away_score'].sum()
teams = teams.join(temp, on='team_id')
teams = teams.rename(columns={"home_score": "awayGetScore", "away_score": "awayLostScore"})

# 隊伍為客場總場次
temp = lastSeason.groupby(['away_team']).size()
temp.index.rename('team_id')
temp = temp.rename('awayMatchCount')
teams = teams.join(temp, on='team_id')

In [47]:
teams.loc[10183]


Unnamed: 0                       11
competition_id                   82
short_name_zh                    热刺
short_name_en     Tottenham Hotspur
market_value              985000000
homeGetScore                     35
homeLostScore                    15
homeMatchCount                   19
awayGetScore                     20
awayLostScore                    34
awayMatchCount                   19
homeAttack                  1.23457
awayAttack                  0.87146
homeDefense                0.705467
awayDefense                 1.52505
Name: 10183, dtype: object

In [48]:
teams.loc[10250]

Unnamed: 0               18
competition_id           82
short_name_zh           埃弗顿
short_name_en       Everton
market_value      506000000
homeGetScore             35
homeLostScore            30
homeMatchCount           19
awayGetScore             25
awayLostScore            24
awayMatchCount           19
homeAttack          1.23457
awayAttack          1.08932
homeDefense        0.881834
awayDefense         1.52505
Name: 10250, dtype: object

In [14]:
# TODO 計算進攻率(上賽季)
# 主隊主場平均進球數 = 主隊主場進球數 / 主場賽事場數。
# 主隊主場平均進球數 / 聯盟賽季主場平均進球數
teams['homeAttack'] =  teams['homeGetScore'] / teams['homeMatchCount'] / seasonHomeAvg
teams['awayAttack'] =  teams['awayGetScore'] / teams['awayMatchCount'] / seasonAwayAvg

# TODO 計算防守率(上賽季)
# 客隊客場平均失球數 = 客隊客場失球數 / 客場賽事場數。
# 客隊客場平均失球數 / 聯盟賽季客場平均失球數
teams['homeDefense'] =  teams['awayGetScore'] / teams['homeMatchCount'] / seasonHomeAvg
teams['awayDefense'] =  teams['homeGetScore'] / teams['awayMatchCount'] / seasonAwayAvg

# 删除有NaN rows
teams = teams.dropna() 

In [19]:
teams[]

,Unnamed: 0,competition_id,short_name_zh,short_name_en,market_value,homeGetScore,homeLostScore,homeMatchCount,awayGetScore,awayLostScore,awayMatchCount,homeAttack,awayAttack,homeDefense,awayDefense
team_id,,,,,,,,,,,,,,,
10022,3,82,沃特福德,Watford,235150000,20.0,19.0,19.0,31.0,20.0,19.0,0.705467,1.350763,1.093474,0.871460
10115,4,82,伯恩茅斯,Bournemouth AFC,334250000,23.0,34.0,19.0,33.0,22.0,19.0,0.811287,1.437908,1.164021,1.002179
10135,5,82,曼联,Manchester United,753250000,27.0,9.0,19.0,26.0,22.0,19.0,0.952381,1.132898,0.917108,1.176471
10136,6,82,纽卡斯尔联,Newcastle United,273580000,32.0,24.0,19.0,41.0,12.0,19.0,1.128748,1.786492,1.446208,1.394336
10176,7,82,西汉姆联,West Ham United,348750000,34.0,26.0,19.0,25.0,31.0,19.0,1.199295,1.089325,0.881834,1.481481
10180,8,82,切尔西,Chelsea,807000000,32.0,30.0,19.0,23.0,27.0,19.0,1.128748,1.002179,0.811287,1.394336
10181,9,82,曼城,Manchester City,1280000000,47.0,21.0,19.0,20.0,24.0,19.0,1.657848,0.871460,0.705467,2.047930
10182,10,82,南安普顿,Southampton,237000000,39.0,22.0,19.0,19.0,20.0,19.0,1.375661,0.827887,0.670194,1.699346
10183,11,82,热刺,Tottenham Hotspur,985000000,35.0,15.0,19.0,20.0,34.0,19.0,1.234568,0.871460,0.705467,1.525054


In [7]:
# TODO 計算可能進球數
# 主隊可能進球數 = 主隊進攻率 * 客隊防守率 * 聯盟主隊平均主場進球數
# 客隊可能進球數 = 客隊進攻率 * 主隊防守率 * 聯盟客隊平均客場進球數

In [8]:
# TODO 使用Poisson進算比分分布

In [9]:
# TODO 使用歷史資料進行驗證

In [10]:
# TODO 結果呈現
# 所有比分發生機率
# 依實際分布情況排序
# 依預估分布情況排序